<a href="https://colab.research.google.com/github/vndee/pytorch-vi/blob/master/chatbot_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CHATBOT
**Tác giả**: [Matthew Inkawhich](https://github.com/MatthewInkawhich)

Trong hướng dẫn này chúng ta sẽ khám phá một ứng dụng thú vị của mô hình seq2seq. Chúng ta sẽ huấn luyện một chatbot đơn giản sử dụng data là lời thoại trong phim từ [Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)

Các mô hình có khả năng đàm thoại là một mảng nghiên cứu đang rất được chú ý của trí tuệ nhân tạo. Chatbot có thể tìm thấy trong rất nhiều sản phẩm tiện ích như bộ phận chăm sóc khách hàng hoặc các dịch vụ tư vấn online. Nhưng con bot này thường thuộc dạng retrieval-based (dựa trên truy xuất), đó là các mô hình mà câu trả lời đã được định sẵn cho mỗi loại câu hỏi nhất định. Dạy một cỗ máy để nó có khả năng đàm thoại với con người một cách tự nhiên vẫn là một bài toán khó và còn xa để đi đến lời giải. Gần đây, đi theo sự bùng nổ của học sâu, các mô hình sinh mạnh mẽ như Google's Neural Conversational Model đã tạo ra một bước nhảy vọt ấn tượng. Trong bài hướng dẫn này, chúng ta sẽ hiện thực một kiểu mô hình sinh như vậy với PyTorch.

![](https://pytorch.org/tutorials/_images/bot.png)

```
> hello?
Bot: hello .
> where am I?
Bot: you re in a hospital .
> who are you?
Bot: i m a lawyer .
> how are you doing?
Bot: i m fine .
> are you my friend?
Bot: no .
> you're under arrest
Bot: i m trying to help you !
> i'm just kidding
Bot: i m sorry .
> where are you from?
Bot: san francisco .
> it's time for me to leave
Bot: i know .
> goodbye
Bot: goodbye .
```

### Các phần chính:
- Load và tiền xử lý [Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) dataset.
- Hiện thực mô hình seq2seq với Luong's attention.
- Phối hợp huấn luyện mô hình encoder-decoder với mini-batches.
- Hiện thực thuật toán decoding bằng tìm kiếm tham lam.
- Tương tác với mô hình đã huấn luyện.

### Lời cảm ơn:
Code trong bài viết này được mượn từ các project mã nguồn mở sau:
- Yuan-Kuei Wu’s pytorch-chatbot implementation: https://github.com/ywk991112/pytorch-chatbot
- Sean Robertson’s practical-pytorch seq2seq-translation example: https://github.com/spro/practical-pytorch/tree/master/seq2seq-translation
- FloydHub’s Cornell Movie Corpus preprocessing code: https://github.com/floydhub/textutil-preprocess-cornell-movie-corpus

## Chuẩn bị
Đầu tiên chúng ta cần tải dữ liệu tại [đây](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) và giải nén.

In [1]:
!wget --header 'Host: www.cs.cornell.edu' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip' --output-document 'cornell_movie_dialogs_corpus.zip'
!unzip cornell_movie_dialogs_corpus.zip

--2019-05-22 05:32:16--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  1.13MB/s    in 8.4s    

2019-05-22 05:32:30 (1.13 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-

In [2]:
!ls cornell\ movie-dialogs\ corpus

chameleons.pdf		       movie_lines.txt		  README.txt
movie_characters_metadata.txt  movie_titles_metadata.txt
movie_conversations.txt        raw_script_urls.txt


Import một số thư viện hỗ trợ:

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

## Load và tiền xử lý dữ liệu
Bước tiếp theo chúng ta cần tổ chức lại dữ liệu. Cornell Movie-Dialogs Corpus là một tập dữ liệu lớn gồm các đoạn hội thoại của các nhân vật trong phim.
- 220,579 đoạn hội thoại của 10,292 cặp nhân vật.
- 9,035 nhân vật từ 617 bộ phim.
- 304,713 cách diễn đạt.

Tập dữ liệu này rất lớn và phân tán, đa dạng trong phong cách ngôn ngữ, thời gian, địa điểm cũng như ý nghĩa. Chúng ta hi vọng mô hình của mình sẽ đủ tốt để làm việc với nhiều cách nói hay truy vấn khác nhau.
Trước hết, hãy xem một vài dòng từ dữ liệu gốc, xem chúng ta có gì ở đây.

In [4]:
corpus_name = 'cornell movie-dialogs corpus'

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus_name, 'movie_lines.txt'))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


Để thuận tiện, chúng ta sẽ tổ chức lại dữ liệu theo một format mỗi dòng trong file sẽ được tách ra bởi dấu tab cho một câu hỏi và một câu trả lời.

Phía dưới chúng ta sẽ cần một số phương thức để phân tích dữ liệu từ file movie_lines.tx
- `loadLines': Tách mỗi dòng dữ liệu thành một đối tượng dictionary trong python gồm các thuộc tính (lineID, characterID, movieID, character, text).
-`loadConversations`: Nhóm các thuộc tính của từng dòng trong `loadLines` thành một đoạn hội thoại dựa trên movie_conversations.txt.
- `extractSentencePairs`: Trích xuất một cặp câu trong đoạn hội thoại.

In [0]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            lineIds = eval(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs


Bây giờ chúng ta sẽ gọi các phương thức ở trên để tạo ra một file dữ liệu mới tên là formatted_movie_lines.txt.

In [6]:
# Define path to new file
datafile = os.path.join(corpus_name, 'formatted_movie_lines.txt')

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus_name, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus_name, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't dat

### Đọc và cắt dữ liệu
Sau khi đã tổ chức lại dữ liệu, chúng ta cần tạo một từ điển các từ dùng trong tập dữ liệu và đọc các cặp câu truy vấn - phản hồi vào bộ nhớ.

Chú ý rằng chúng ta xem một câu là một chuỗi liên tiếp các **từ**, không có một ánh xạ ngầm nào của nó ở một không gian số học rời rạc. Do đó chúng ta cần phải tạo một hàm ánh xạ sao cho mỗi từ riêng biệt chỉ có duy nhất một giá trị chỉ số đại diện chính là vị trí của nó trong từ điển.

Để làm điều đó chúng ta định nghĩa lớp `Voc`, nơi sẽ lưu một dictionary ánh xạ **từ** sang **chỉ số**, một dictionary ánh xạ ngược **chỉ số** sang **từ**, một biến đếm cho mỗi từ và một biến đếm tổng số các từ. Lớp `Voc` cũng cung cắp các phương thức để thêm một từ vào từ điển (`addWord`), thêm tất cả các từ trong một câu (`addSentence`) và lược bỏ (trimming) các từ không thường gặp. Chúng ta sẽ nói về trimming sau:


In [0]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Trước khi đưa vào huấn luyện ta cần một số thao tác tiền xử lý dữ liệu. Đầu tiên, chúng ta cần chuyển đổi các chuỗi Unicode thành ASCII sử dụng `unicodeToAscii`. Tiếp theo phải chuyển tất cả các kí tự thành chữ viết thường và lược bỏ các kí tự không ở trong bảng chữ cái ngoại trừ một số dấu câu (`normalizedString`). Cuối cùng để giúp quá trình huấn luyện nhanh chóng hội tụ chúng ta sẽ lọc ra các câu có độ dài lớn hơn ngưỡng `MAX_LENGTH` (`filterPairs`).

In [8]:
MAX_LENGTH = 10  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("save")
voc, pairs = loadPrepareData(corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


Một chiến thuật khác để giúp mô hình học nhanh hơn đó là lược bỏ các từ hiếm gặp trong dữ liệu. Việc này giúp làm giảm đi độ khó của bài toán, và do đó mô hình sẽ hội tụ nhanh hơn. Chúng ta sẽ làm điều này bằng 2 bước.
- Lược bỏ các từ với tần suất xuất hiện ít hơn `MIN_COUNT` sử dụng phương thức `voc.trim`.
- Lược bỏ các cặp câu hội thoại có chứa từ bị cắt ở bước trên.


In [9]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


## Chuẩn bị dữ liệu cho mô hình

Mặc dù ở trên chúng ta đã làm rất nhiều thứ để có một bộ dữ liệu tốt gồm các cặp câu hội thoại, từ điển. Nhưng mô hình của chúng ta luôn mong đợi dữ liệu vào của nó phải là numerical torch tensor. Cách để chuyển dữ liệu dạng này thành tensor có thể tìm thấy ở bài viết [seq2seq translation tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html). Trong bài viết này chúng ta chỉ dùng batch size bằng 1, tất cả những gì chúng ta phải làm là chuyển tất cả các từ trong một cặp câu thành chỉ số tương ứng của nó trong từ điển và đưa vào mô hình huấn luyện.

Tuy nhiên, nếu muốn quá trình huấn luyện nhanh hơn và tận dụng được khả năng tính toán song song của GPU chúng ta nên huấn luyện theo mini-batches.

Sử dụng mini-batches thì cần phải chú ý rằng các câu trong một batch có thể sẽ có độ dài không giống nhau. Vì vậy chúng ta nên đặt số chiều của các tensor batch cố định là (max_length, batch_size). Các câu có độ dài nhỏ hơn max_length sẽ được thêm zero padding phía sau kí tự EOS_token (kí tự kết thúc câu).

Một vấn đề khác đặt ra là nếu chúng ta chuyển tất cả các từ của một cặp câu vào một batch tensor, lúc này tensor của chúng ta sẽ có kích thước là (max_length, batch_size). Tuy nhiên cái chúng ta cần là một tensor với kích thước (batch_size, max_length) và lúc đó cần phải hiện thực thêm một phướng thức để chuyển vị ma trận. Thay vì rườm ra như vậy, chúng ta sẽ thực hiện việc chuyển vị đó ngay từ trong hàm `zeroPadding`.

![](https://pytorch.org/tutorials/_images/seq2seq_batches.png)


In [16]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    
    return m

# Returns padded input sequene tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(' ')), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
        
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print('input_variable:', input_variable)
print('lengths:', lengths)
print('target_variable:', target_variable)
print('mask:', mask)
print('max_target_len:', max_target_len)

input_variable: tensor([[ 112,  726,    5,  242,   48],
        [  12, 3610,   37,  188, 1626],
        [ 130,    4,   53,   45,    4],
        [   4,  758,  606,  140,    2],
        [  34, 1205,    6,    4,    0],
        [   7,    4,    2,    2,    0],
        [ 197,    2,    0,    0,    0],
        [ 117,    0,    0,    0,    0],
        [   4,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([10,  7,  6,  6,  4])
target_variable: tensor([[ 27,  25,   7, 167,  25],
        [ 14, 296, 118,   6, 112],
        [ 67,  66,  70,   2,  94],
        [123,  25, 606,   0, 117],
        [ 21, 296,   6,   0,   4],
        [ 22,  66,   2,   0,   2],
        [  4,   2,   0,   0,   0],
        [  2,   0,   0,   0,   0]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 0, 0, 0],
        [1, 0, 0, 0, 0]], dtype=torch.uint8)
max_target_len: 8


##Định nghĩa mô hình
###Mô hình Seq2Seq

Bộ não chatbot của chúng ta là một mô hình sequence-to-sequence (seq2seq). Mục tiêu của mô hình seq2seq là nhận một chuỗi đầu vào và dự đoán chuỗi đầu ra dựa trên mô mô hình cố định.

[Sutskever và các cộng sự](https://arxiv.org/abs/1409.3215) đã đề xuất một phương pháp dựa trên hai mô hình mạng nơ-ron hồi quy (RNN) có thể giải quyết được bài toán này. Một RNN hoạt động như một encoder (bộ mã hóa), encoder có nhiệm vụ mã hóa chuỗi đầu vào thành một context vector (vector ngữ cảnh). Trên lý thuyết, context vector (layer cuối cùng của RNN) sẽ chứa các thông tin ngữ nghĩa của chuỗi đầu vào. RNN thứ hai là decoder (bộ giải mã), nó dùng context vector của encoder để dự đoán chuỗi đầu ra tương ứng.

![](https://pytorch.org/tutorials/_images/seq2seq_ts.png)

*Nguồn ảnh: https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_intro/*

###Encoder

Bộ mã hóa sử dụng mạng nơ-ron hồi quy (encoder RNN) duyệt qua từng token của chuỗi đầu vào, tại mỗi thời điểm xuất ra một "output" vector và một "hidden state" vector. Hidden state vector sau đó sẽ được dùng để tính hidden state vector tại thời điểm tiếp theo như trong ý tưởng cơ bản của RNN. Mạng encoder sẽ cố gắn g chuyển đổi những cái gì nó nhìn thấy trong chuỗi đầu vào bao gồm cả ngữ cảnh và ngữ nghĩa thành một tập hợp các điểm trong một không gian nhiều chiều, nơi decoder nhìn vào để giải mã chuỗi đầu ra có ý nghĩa.

Trái tim của encoder là multi-layered Gate Recurrent Unit, được đề xuất bởi [Cho và các cộng sư](https://arxiv.org/pdf/1406.1078v3.pdf) vào năm 2014. Chúng ta sẽ dùng dạng hai chiều của GRU, đồng nghĩa với việc có 2 mạng RNN độc lập: một đọc chuỗi đầu vào theo một thứ tự từ trái sáng phải, một từ phải sang trái.

![](https://pytorch.org/tutorials/_images/RNN-bidirectional.png)

*Nguồn ảnh: https://colah.github.io/posts/2015-09-NN-Types-FP/*

Chú ý rằng `embedding` layer được dùng để mã hóa từng từ trong câu văn đầu vào thành một vector trong không gian ngữ nghĩa của nó.

Cuối cùng, nếu đưa một batch dữ liệu vào RNN, chúng ta cần phải "unpack" zeros padding xung quanh của từng chuỗi. 

####Các bước tính toán
1. Chuyển word index thành embedding vector.
2. Đóng gói các câu thành một các batch.
3. Đưa từng batch qua GRU để tính toán.
4. Unpack padding.
5. Cộng tất cả các output của GRU hai chiều.
6. Trả về kết quả và hidden state cuối cùng.

####Input:
- `input_seq`: batch of input sentences, kích thước (max_length, batch_size)
- `input_lengths`: Danh sách chứa độ dài câu tương ứng với từng câu trong batch, kích thước (batch_size)
- `hidden`: hidden state, kích thước (n_layers * num_directions, batch_size, hidden_size)

####Output:
- `output`: Layer của cuối cùng của GRU, kích thước (max_length, batch_size, hidden_size)
- `hidden`: cập nhật hidden state từ GRU, kích thước (n_layers * num_directions, batch_size, hidden_size)

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        
        # Initialize GRU; the input_size and hidden_size params are both set to 
        # 'hidden_size' because our input size is a word embedding with number 
        # of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                         dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embedding vector
        embedded = self.embedding(input_seq)
        
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        
        # Sum bidirectional GRU outputs
        output = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        
        # Return output and final hidden state
        return outputs, hidden

###Decoder
Bộ giải mã RNN sẽ sinh ra chuỗi đầu ra theo từng token. Nó sử dụng context vector của encoder và hidden state để sinh từ tiếp theo trong chuỗi đầu ra cho đến khi gặp phải EOS_token (kí hiệu kết thúc câu).  Một vấn đề với bài toán seq2seq truyền thống đó là nếu chỉ dùng context vector và hidden state thì sẽ bị mất mát thông tin, đặc biệt là với những câu dài.

Để đối phó với điều đó, [Bahdanau](https://arxiv.org/abs/1409.0473) đã đề xuất một phương pháp gọi là cơ chế attention. Cơ chế này cho phép decoder đặt sự chú ý lên một vài điểm nhất định trong câu thay vì nhìn các từ với mức độ quan trọng y như nhau. 

Attention được tính toán dựa vào hidden state hiện tại của decoder và kết quả của encoder. Bộ trọng số của attention có cùng kích thước với chuồi đầu vào.

![](https://pytorch.org/tutorials/_images/attn2.png)

[Luong](https://arxiv.org/abs/1508.04025) attention là một phiên bản cải tiến với ý tưởng "Global attention". Sự khác biệt là với "Global attention" chúng ta sẽ nhìn tất cả các hidden state của encoder, thay vì chỉ nhìn hidden state cuối cùng của encoder như của Bahdanau. Một khác biệt nữa là "global attention" tính dựa trên duy nhất hidden state hiện tại của decoder chứ không như phiên bản của Bahdanau cần phải tính qua hidden state tại các bước trước đó.

![](https://pytorch.org/tutorials/_images/scores.png)

Trong đó: $h_{t}$ là hidden state hiện tại của decoder và $h_{s}$ là toàn bộ hidden state của encoder.

Nhìn chung, global attention có thể tổng hợp như hình bên dưới.

![](https://pytorch.org/tutorials/_images/global_attn.png)

In [0]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, 'is not an appropriate attention method.')
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
            
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))
            
    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_ouput, dim=2)
    
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)
    
    def concat_score(self, hidden, encoder_outputs):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1),
                                     encoder_ouputs), 2)).tanh()
        
        return torch.sum(self.v * energy, dim=2)
    
    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
            
        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()
        
        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

####Các bước tính toán
1. Lấy embedding vector của từ hiện tại
2. Đưa dữ liệu qua GRU hai chiều để tính toán
3. Tính trọng số attention từ output của GRU
4. Nhân trọng số của attention của encoder output để có được trọng số mới của context vector.
5. Nối (concat) context vector và GRU hidden state như trong công thức của Luong attention.
6. Dự đoán từ tiếp theo dựa trên Luong attention
7. Trả về kết quả và hidden state cuối cùng

####Inputs:
- `input_step`: Một step là một đơn vị thời gian, kích thước (1, batch_size)
-  `last_hidden`: hidden layer cuối của GRU, kích thước (n_layers * num_directión, batch_size, hidden_size)
-  `encoder_outputs`: encoder output, kích thước (max_length, batch_size, hidden_size)

####Outputs:
- `output`: softmax normalized tensor, kích thước (batch_size, voc.num_words)
- `hidden`: hidden state cuối của GRU, kích thước (n_layers * num_directions, batch_size, hidden_size)

In [0]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        
        # Return output and final hidden state
        return output, hidden

##Huấn luyện
###Masked loss
Vì chúng ta đang làm việc với batch of padded sentences, cho nên không thể dễ dàng để tính loss cho tất cả các thành phần của tensor. Chúng ta định nghĩa hàm `maskNLLLoss` để tính loss dựa trên output của decoder. Kết quả trả về là trung bình negative log likelihood của các thành phần trong tensor (mỗi thành phần là một câu).


In [0]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -troch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_selected(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

###Training
Hàm `train` hiện thực thuật toán huấn luyện cho một lần lặp.
Chúng ta sẽ dùng một vài kỹ thuật để quá trình training diễn ra tốt hơn:
- **Teacher forcing**: Kỹ thuật này cho phép với một xác suất được quy định sẵn `teacher_forcing_ratio`, decoder sẽ dùng target word tại thời điểm hiện tại để dự đoán từ tiếp theo thay vì dùng từ được dự đoán bởi decoder tại thời điểm hiện tại.
- **Gradient clipping**: Đây là một kỹ thuật thường dùng để đối phố với "exploding gradient".  Kỹ thuật này đơn giản là chặn giá trị gradient ở một ngưỡng trên, không để nó trở nên quá lớn.

![](https://pytorch.org/tutorials/_images/grad_clip.png)
*Nguồn ảnh: Goodfellow et al. Deep Learning. 2016. https://www.deeplearningbook.org/*

####Các bước tính toán
1. Đưa toàn bộ batch vào encoder đê tính toán.
2. Khởi tạo input cho decoder bằng SOS_token và hidden state bằng với hidden state cuối cùng của encoder.
3. Đưa chuỗi input qua decoder.
4. If teacher_forcing: gán input tại thời điểm tiếp theo của decoder bằng nhãn đúng của từ dự đoán hiện tại, ngược lại gán bằng từ được decoder dự đoán tại thời điểm hiện tại.
5. Tính loss
6. Thực hiện giải thuật lan truyền ngược.
7. Clip gradients.
8. Cập nhật trọng số encoder và decoder.



In [0]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals